In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#from lxml import html

In [12]:
def getGradeSource(URL):
    arrTotal = []

    quote_page = URL
    #URL of gradesource website that needs to be scraped
    page = requests.get(quote_page)
    #making a request
    soup = BeautifulSoup(page.content, 'html.parser')
    #cleaning the html
    
    table = soup.find_all('tr')
    #retrieving all the table
    arr = []

    for i in range(len(table)):
        arr.append(table[i].find_all('td'))

    for i in range(len(arr)):
        if i >= 5 and i < len(arr):     
            arrHolder = []
            for k in range(len(arr[i])):
                if arr[i][k].find("b") != None:
                    arrHolder.append(arr[i][k].find("b").text) # getting the course id
                elif arr[i][k].find("font") != None:
                    arrHolder.append(arr[i][k].find("font").text) # getting the rank
            arrTotal.append(arrHolder)
    return arrTotal

In [13]:
CSE110 = getGradeSource('http://www.gradesource.com/reports/7/29889/cs126276.html')
cse15L = getGradeSource('http://www.gradesource.com/reports/7/29142/categoryscores.html')

In [16]:
for i in CSE110:
    print(i)

['Secret Number', 'Participation', 'Teamwork', 'Labs', 'Project Artifacts', 'Quizzes(Best 1 out of 2)', 'Course']
['\xa0', 'Rank', 'Points', 'Score', 'Rank', 'Points', 'Score', 'Rank', 'Points', 'Score', 'Rank', 'Points', 'Score', 'Rank', 'Score', 'Rank', 'Score']
['\xa0', '\xa0', '30', '100%', '\xa0', '35', '100%', '\xa0', '20', '100%', '\xa0', '30', '100%', '\xa0', '100%', '\xa0', '100%']
['8605', '1', '30', '100.00%', '1', '35', '100.00%', '1', '20', '100.00%', '1', '30', '100.00%', '161', '85.71%', '81', '92.64%']
['4605', '1', '30', '100.00%', '1', '35', '100.00%', '1', '20', '100.00%', '1', '30', '100.00%', '69', '92.31%', '22', '96.04%']
['6671', '1', '30', '100.00%', '1', '35', '100.00%', '1', '20', '100.00%', '1', '30', '100.00%', '161', '85.71%', '81', '92.64%']
['0273', '170', '29', '96.67%', '1', '35', '100.00%', '1', '20', '100.00%', '1', '30', '100.00%', '26', '96.15%', '15', '97.66%']
['3243', '1', '30', '100.00%', '1', '35', '100.00%', '1', '20', '100.00%', '1', '30', '

In [17]:
for i in cse15L:
    print(i)

['Secret Number', 'Labs(Best 8 out of 9)', 'Homework', 'Scripting Project', 'Midterm Exam', 'Final Exam', 'Professionalism', 'Course']
['\xa0', 'Rank', 'Points', 'Score', 'Rank', 'Points', 'Score', 'Rank', 'Points', 'Score', 'Rank', 'Points', 'Score', 'Rank', 'Points', 'Score', 'Rank', 'Points', 'Score', 'Rank', 'Score']
['\xa0', '\xa0', '80', '100%', '\xa0', '50', '100%', '\xa0', '10', '100%', '\xa0', '116', '100%', '\xa0', '162', '100%', '\xa0', '1', '100%', '\xa0', '100%']
['0038', '1', '80', '100.00%', '244', '38', '76.00%', '1', '10', '100.00%', '187', '91.75', '79.09%', '245', '121', '74.69%', '1', '1', '100.00%', '210', '85.61%']
['0071', '328', '70', '87.50%', '292', '34.6', '69.20%', '1', '10', '100.00%', '213', '89', '76.72%', '272', '115.5', '71.30%', '1', '1', '100.00%', '277', '79.76%']
['0077', '1', '80', '100.00%', '96', '44.5', '89.00%', '350', '3.5', '35.00%', '327', '71.25', '61.42%', '308', '108.25', '66.82%', '1', '1', '100.00%', '317', '75.00%']
['0154', '1', '80',

In [7]:
AutograderURL = 'https://autograder.ucsd.edu/login'
autoGraderPage = requests.get(AutograderURL)
prettySoup = BeautifulSoup(autoGraderPage.content, 'html.parser')

In [8]:
prettySoup.prettify()

'<!DOCTYPE HTML>\n<html>\n <head>\n  <title>\n   Login - Autograder\n  </title>\n  <meta charset="utf-8"/>\n  <meta content="true" name="HandheldFriendly">\n   <meta content="width=device-width, minimum-scale=.85, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>\n   <link href="/public/images/favicon.ico" rel="icon" type="image/x-icon"/>\n   <link href="/public/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>\n   <link charset="utf-8" href="/public/stylesheets/bootstrap.min.css" rel="stylesheet" type="text/css"/>\n   <link href="https://gitcdn.github.io/bootstrap-toggle/2.2.2/css/bootstrap-toggle.min.css" rel="stylesheet"/>\n   <link charset="utf-8" href="/public/stylesheets/bootstrap-nav.css" rel="stylesheet" type="text/css"/>\n   <link charset="utf-8" href="/public/stylesheets/bootstrap-override.css" rel="stylesheet" type="text/css"/>\n   <link charset="utf-8" href="/public/stylesheets/font-awesome.min.css" rel="stylesheet" type="text/css"/>\n   <

In [9]:
inputList = prettySoup.find_all('input')

In [11]:
for i in inputList:
    print(i)

<input name="authenticityToken" type="hidden" value="6bfb189422a2951cf3000809075ed9e34745d966"/>
<input class="form-control" id="email" name="username" placeholder="Email" type="email"/>
<input class="form-control" id="password" name="password" placeholder="Password" type="password"/>


In [12]:
tree = html.fromstring(prettySoup.text)
authenticity_token = list(set(tree.xpath("//input[@name='username']/@value")))[0]

result = autoGraderPage.post(
    login_url, 
    data = payload, 
    headers = dict(referer=login_url)
)

NameError: name 'html' is not defined